# 03: Spatial Analysis — Narrative Density Mapping

**How RPG Maker's Map Architecture Structures Storytelling**

This notebook explores how narrative content is distributed across the game's spatial structure.

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from experiment_2_density import NarrativeDensity
from visualization import setup_style, COLORS

setup_style()

## 1. Load and Parse Map Data

In [ ]:
# Initialize density analysis
density = NarrativeDensity('../data/EventTextDump.txt')
density.parse_data()
density.count_commands_by_map()

print(f"Maps parsed: {len(density.map_data)}")
print(f"Maps with content: {len(density.density_df)}")

## 2. Density Distribution

In [ ]:
# Calculate statistics
stats = density.calculate_statistics()

print("Narrative Density Statistics:")
print(f"  Mean: {stats['mean_density']:.3f}")
print(f"  Median: {stats['median_density']:.3f}")
print(f"  Std Dev: {stats['std_density']:.3f}")
print(f"\nClassification:")
print(f"  Story-heavy (>0.7): {stats['story_heavy_count']} ({stats['story_heavy_pct']:.1f}%)")
print(f"  Mixed (0.3-0.7): {stats['mixed_count']} ({stats['mixed_pct']:.1f}%)")
print(f"  Combat/Traversal (<0.3): {stats['combat_count']} ({stats['combat_pct']:.1f}%)")

In [ ]:
# Histogram of density values
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(density.density_df['narrative_density'], bins=30, 
        color=COLORS['story'], edgecolor='white', alpha=0.7)
ax.axvline(0.3, color=COLORS['combat'], linestyle='--', label='Combat threshold (0.3)')
ax.axvline(0.7, color=COLORS['story'], linestyle='--', label='Story threshold (0.7)')
ax.set_xlabel('Narrative Density (dialogue/total commands)')
ax.set_ylabel('Number of Maps')
ax.set_title('Distribution of Narrative Density Across Game Maps')
ax.legend()
plt.tight_layout()
plt.show()

## 3. Top Story Locations

In [ ]:
# Top 10 story-rich maps
top_story = density.density_df.nlargest(10, 'narrative_density')[['map_name', 'dialogue_count', 'total_commands', 'narrative_density', 'classification']]
print("Top 10 Narrative-Dense Maps:")
print(top_story.to_string(index=False))

In [ ]:
# Visualize top story maps
fig, ax = plt.subplots(figsize=(10, 6))
colors = [COLORS['story'] if c == 'Story-Heavy' else COLORS['mixed'] if c == 'Mixed' else COLORS['combat'] 
          for c in top_story['classification']]
bars = ax.barh(range(len(top_story)), top_story['narrative_density'], color=colors)
ax.set_yticks(range(len(top_story)))
ax.set_yticklabels(top_story['map_name'])
ax.invert_yaxis()
ax.set_xlabel('Narrative Density')
ax.set_title('Top 10 Story-Rich Locations')
plt.tight_layout()
plt.show()

## 4. Combat Zones

In [ ]:
# Top 10 combat-heavy maps (by total commands, low density)
combat_maps = density.density_df[density.density_df['classification'] == 'Combat/Traversal']
top_combat = combat_maps.nlargest(10, 'total_commands')[['map_name', 'dialogue_count', 'total_commands', 'narrative_density']]
print("Top 10 Combat/Traversal Zones (by activity):")
print(top_combat.to_string(index=False))

## 5. Classification Breakdown

In [ ]:
# Pie chart of classifications
class_counts = density.density_df['classification'].value_counts()

fig, ax = plt.subplots(figsize=(8, 8))
colors_pie = [COLORS['combat'], COLORS['mixed'], COLORS['story']]
ax.pie(class_counts.values, labels=class_counts.index, autopct='%1.1f%%', colors=colors_pie)
ax.set_title('Map Classification Distribution')
plt.show()

## 6. Command Type Analysis by Classification

In [ ]:
# Average command composition by classification
cmd_cols = [c for c in density.density_df.columns if c not in ['map_id', 'map_name', 'dialogue_count', 'total_commands', 'narrative_density', 'classification']]

if cmd_cols:
    avg_by_class = density.density_df.groupby('classification')[cmd_cols].mean()
    print("Average Command Counts by Classification:")
    print(avg_by_class.T.to_string())

## 7. Spatial Dramaturgy Interpretation

The data reveals a **hub-and-spoke** narrative architecture:

- **83% combat/traversal zones**: Most of the game world is dangerous, resource-depleting space
- **17% mixed zones**: Narrative hubs where story content concentrates
- **0% pure story zones**: Even the most dialogue-rich areas include gameplay commands

This mirrors *Ballad of a Soldier*'s central tension: war is endless, brutal traversal punctuated by brief moments of human connection.

---

**Next**: Proceed to `04_network_exploration.ipynb` to explore the semantic network analysis.